## Исследование надёжности заёмщиков

Кредитному отделу банка необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Банк предоставил статистику о платёжеспособности клиентов. Результаты анализа будут использованы при построении модели **кредитного скоринга** — системы, которая оценивает способность потенциального заёмщика выплатить кредит.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd


data = pd.read_csv('/datasets/data.csv')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

В столбцах days_employed и total_income есть пропуски, причём скорее всего в строках с одними и теми же данными. Это значит, что банк не обладает данными о стаже работы и доходах более 2000 своих заёмщиков. Столбец days_employed имеет тип float64, что явно является недочётом составителей или ошибкой чтения. Пока непонятно, почему долг измеряется целыми числами, а доход - дробными.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#заменяем пропуcки на средние или медианные значения

#обработаем столбцы total_income и days_employed

#для типа "сотрудник"
employee = data[data['income_type'] == 'сотрудник']
employee_income_mean = employee.loc[:,'total_income'].mean()
employee.loc[:,'total_income'] = employee.loc[:,'total_income'].fillna(value=employee_income_mean)
employee_days_mean = employee.loc[:,'days_employed'].mean()
employee.loc[:,'days_employed'] = employee.loc[:,'days_employed'].fillna(value=employee_days_mean)
data[data['income_type'] == 'сотрудник'] = employee


#для типа "компаньон"
compangion = data[data['income_type'] == 'компаньон']
compangion_income_median = compangion.loc[:, 'total_income'].median()
compangion.loc[:, 'total_income'] = compangion.loc[:, 'total_income'].fillna(value=compangion_income_median)
compangion_days_mean = compangion.loc[:, 'days_employed'].mean()
compangion.loc[:, 'days_employed'] = compangion.loc[:, 'days_employed'].fillna(value=compangion_days_mean)
data[data['income_type'] == 'компаньон'] = compangion

#для типа "пенсионер"
pensioner = data[data['income_type'] == 'пенсионер']
pensioner_income_median = pensioner.loc[:, 'total_income'].median()
pensioner.loc[:, 'total_income'] = pensioner.loc[:, 'total_income'].fillna(value=pensioner_income_median)
pensioner_days_mean = pensioner.loc[:, 'days_employed'].mean()
pensioner.loc[:, 'days_employed'] = pensioner.loc[:, 'days_employed'].fillna(value=pensioner_days_mean)
data[data['income_type'] == 'пенсионер'] = pensioner


#для типа "госслужащий"
civil_service = data[data['income_type'] == 'госслужащий']
cs_income_median = civil_service.loc[:, 'total_income'].median()
civil_service.loc[:, 'total_income'] = civil_service.loc[:, 'total_income'].fillna(value=cs_income_median)
cs_days_mean = civil_service.loc[:, 'days_employed'].mean()
civil_service.loc[:, 'days_employed'] = civil_service.loc[:, 'days_employed'].fillna(value=cs_days_mean)
data[data['income_type'] == 'госслужащий'] = civil_service 


#для типа "предприниматель"
entrepreneur = data[data['income_type'] == 'предприниматель']
entrepreneur_income = entrepreneur.loc[:, 'total_income'].mean()
entrepreneur.loc[:, 'total_income'] = entrepreneur.loc[:, 'total_income'].fillna(value=entrepreneur_income)
entrepreneur_days = entrepreneur.loc[:, 'days_employed'].mean()
entrepreneur.loc[:, 'days_employed'] = entrepreneur.loc[:, 'days_employed'].fillna(value=entrepreneur_income)
data[data['income_type'] == 'предприниматель'] = entrepreneur


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Вывод

В столбцах, содержащих информацию о совокупном доходе и трудовом стаже, встречаются пропуски. Они были заменены на средние или медианные значения в зависимости от типа трудоустройства. Медианные значения использовались для тех категорий, которые имели большой разрыв в зарплате между минимальным и максимальным значениями.

### Замена типа данных

In [3]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

### Вывод

В столбцах, содержащих информацию о стаже работы и совокупном доходе, вещественный тип данных был заменён на целочисленный, так как в таком виде данные легче воспринимаются человеком.

### Обработка дубликатов

In [4]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)

data['education'] = data['education'].str.lower()
data = data.drop_duplicates().reset_index(drop=True)

### Вывод

В столбце с указанием уровня образования были обнаружены одинаковые значения, написанные в разном регистре. Для того, чтобы гарантировать удаление всех дубликатов, все значения указанного столбца были приведены к нижнему регистру. Далее были удалены все дублирующиеся строки данных.

### Лемматизация

In [5]:
from pymystem3 import Mystem
from collections import Counter 


m = Mystem()
  
purposes = data['purpose']    
list_of_lemmas = []    

for item in purposes:
    lemmas = m.lemmatize(item)
    for lemma in lemmas:
        list_of_lemmas.append(lemma)

lemmas_count = Counter(list_of_lemmas)

## Категоризация данных

In [6]:
from pymystem3 import Mystem
from collections import Counter 


m = Mystem()
def categorize_purpose(line):
    common_purposes = ['жилье', 'коммерческий', 'недвижимость', 'автомобиль', 'образование', 'свадьба']
    lemma = m.lemmatize(line)
    for purpose in common_purposes:
        if purpose in lemma:
            return purpose
                
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data['purpose_category'] = data['purpose_category'].replace('коммерческий', 'коммерция')

def categorize_income(income_data):
    if income_data <= 120000:
        return 'низкий'
    elif 120000 < income_data <= 600000:
        return 'средний'
    elif 600000 < income_data <= 1200000:
        return 'выше среднего'
    elif 1200000 < income_data:
        return 'высокий'
    
data['income_size'] = data['total_income'].apply(categorize_income)

### Вывод

В результате лемматизации и последующего подсчёта наиболее встречающихся лемм, было выявлено, что цели кредита обобщённо можно разделить на: жилую недвижимость, коммерческую недвижимость, автомобиль, образование и свадьбу. Чтобы первые две цели сформулировать короче, было решено оставить от первой "жилье", а от второй - "коммерция". Категории целей вынесены в отдельный столбец таблицы.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
debt_and_children = data.loc[:, ['children', 'debt']]

family_debt = len(debt_and_children[(debt_and_children['debt'] > 0) & (debt_and_children['children'] > 0)])
childfree_debt = len(debt_and_children[(debt_and_children['debt'] > 0) & (debt_and_children['children'] == 0)])

family = len(debt_and_children[debt_and_children['children'] > 0])
childfree = len(debt_and_children[debt_and_children['children'] == 0])
total = family + childfree

family_part = family_debt / total
childfree_part = childfree_debt / total

print('Доля должников с детьми: {:.0%}'.format(family_part))
print('Доля должников без детей: {:.0%}'.format(childfree_part))

Доля должников с детьми: 3%
Доля должников без детей: 5%


### Вывод

Как показывает проведённый анализ, наличие детей не сильно влияет на процент должников по кредиту (разница = 2%).

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
status_and_debt = data.loc[:, ['family_status', 'family_status_id', 'debt']]

married = status_and_debt[status_and_debt['family_status_id'] == 0]
civil_marriage = status_and_debt[status_and_debt['family_status_id'] == 1]
widow = status_and_debt[status_and_debt['family_status_id'] == 2]
divorced = status_and_debt[status_and_debt['family_status_id'] == 3]
not_married = status_and_debt[status_and_debt['family_status_id'] == 4]

married_debt = len(married[married['debt'] > 0])
civil_marriage_debt = len(civil_marriage[civil_marriage['debt'] > 0])
widow_debt = len(widow[widow['debt'] > 0])
divorced_debt = len(divorced[divorced['debt'] > 0])
not_married_debt = len(not_married[not_married['debt'] > 0])

total_debt = len(data[data['debt'] > 0])

married_part = married_debt / total_debt
civil_marriage_part = civil_marriage_debt / total_debt
widow_part = widow_debt / total_debt
divorced_part = divorced_debt / total_debt 
not_married_part = not_married_debt / total_debt

list_of_status = data['family_status'].unique()
#print(list_of_status)['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе','Не женат / не замужем']

percent=[married_part, civil_marriage_part, widow_part, divorced_part, not_married_part]
header = list_of_status

print('Доля должников')
print()
print('| ', end='')
for name in header:
    print(name, end=' | ')
print()
print('-------------------------------------------------------------------------------------------')

print(' {: 16.0%} | {: 16.0%} | {: 14.0%} | {: 9.0%} | {: 22.0%}'.format(percent[0], percent[1], percent[2], percent[3],
                                                                               percent[4]))

Доля должников

| женат / замужем | гражданский брак | вдовец / вдова | в разводе | Не женат / не замужем | 
-------------------------------------------------------------------------------------------
              53% |              22% |             4% |        5% |                    16%


### Вывод

Среди должников больше всего людей, состоящих в официальном браке (53%).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
income_and_debt = data.loc[:, ['income_size', 'debt']]

low = income_and_debt[income_and_debt['income_size'] == 'низкий']
medium = income_and_debt[income_and_debt['income_size'] == 'средний']
medium_plus = income_and_debt[income_and_debt['income_size'] == 'выше среднего']
high = status_and_debt[income_and_debt['income_size'] == 'высокий']

low_debt = len(low[low['debt'] > 0])
medium_debt = len(medium[medium['debt'] > 0])
medium_plus_debt = len(medium_plus[medium_plus['debt'] > 0])
high_debt = len(high[high['debt'] > 0])

low_part = low_debt / total_debt
medium_part = medium_debt / total_debt
medium_plus_part = medium_plus_debt / total_debt
high_part = high_debt / total_debt

income_header = data['income_size'].unique()
values = [low_part, medium_part, medium_plus_part, high_part]

print('Доля должников')
print()
print('| ', end='')
for cat in income_header:
    print(cat, end=' | ')
print()
print('-------------------------------------------------------------------------------------------')

print(' {: 8.1%} | {: 6.1%} | {: 13.1%} | {: 8.1%} '.format(values[1], values[0], values[2], values[3]))


Доля должников

| средний | низкий | выше среднего | высокий | 
-------------------------------------------------------------------------------------------
    65.9% |  33.7% |          0.4% |     0.1% 


### Вывод

Среди должников около 66% клиентов со средним доходом (до 50 000 в месяц). Треть должников имеют средний доход (меньше прожиточного минимума). Следовательно, средний уровень дохода увеличивает вероятность невозврата кредита.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
purpose_and_debt = data.loc[:, ['purpose_category', 'debt']]

commercial = purpose_and_debt[purpose_and_debt['purpose_category'] == 'коммерция']
premises = purpose_and_debt[purpose_and_debt['purpose_category'] == 'недвижимость']
housing = purpose_and_debt[purpose_and_debt['purpose_category'] == 'жилье']
auto = purpose_and_debt[purpose_and_debt['purpose_category'] == 'автомобиль']
education = purpose_and_debt[purpose_and_debt['purpose_category'] == 'образование']
wedding = purpose_and_debt[purpose_and_debt['purpose_category'] == 'свадьба']

commercial_debt = len(commercial[commercial['debt'] > 0])
premises_debt = len(premises[premises['debt'] > 0])
housing_debt = len(housing[housing['debt'] > 0])
auto_debt = len(auto[auto['debt'] > 0])
education_debt = len(education[education['debt'] > 0])
wedding_debt = len(wedding[wedding['debt'] > 0])

commercial_part = commercial_debt / total_debt
housing_part = housing_debt / total_debt
premises_part = premises_debt / total_debt
auto_part = auto_debt / total_debt
education_part = education_debt / total_debt
wedding_part = wedding_debt / total_debt

purpose_header = data['purpose_category'].unique()
cells = [commercial_part, housing_part, premises_part, auto_part, education_part, wedding_part]

print('Доля должников')
print()
print('| ', end='')
for purpose in purpose_header:
    print(purpose, end=' | ')
print()
print('-------------------------------------------------------------------------')

print(' {: 6.0%} | {: 10.0%} | {: 11.0%} | {: 7.0%} | {: 12.0%} | {: 10.0%}'.format(cells[0], cells[1], cells[2], cells[3], 
                                                                                    cells[4], cells[5]))


Доля должников

| жилье | автомобиль | образование | свадьба | недвижимость | коммерция | 
-------------------------------------------------------------------------
     6% |        18% |         22% |     23% |          21% |        11%


### Вывод

Самый высокий процент задолженности у тех, кто брал кредит на свадьбу. Не отстают и кредиты на образование и недвижимость (без указания её специфики). Выводы о влиянии цели кредита на его погашение могут оказаться неточными, так как данные о недвижимости не конкретны. 

### Шаг 4. Общий вывод

По результатам проведённого анализа можно сделать вывод о том, что чаще всего не отдают кредит люди со средним доходом, состоящие в официальном браке и взявшие кредит на свадьбу.